In [1]:
import os

In [2]:
%pwd

'c:\\Users\\assi01\\Desktop\\projects\\AirTravel_Sentiment_Analysis\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\assi01\\Desktop\\projects\\AirTravel_Sentiment_Analysis'

In [49]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=False)
class ModelTrainingConfig:
    root_dir: Path
    base_model_path: Path
    base_tokenizer_path: Path
    model_path: Path
    tokenizer_path: Path
    train_tokenized_data_path: Path
    test_tokenized_data_path: Path
    val_tokenized_data_path: Path
    params_model_name: str
    params_evaluation_strategy: str
    params_save_strategy: str
    params_learning_rate: float
    params_per_device_train_batch_size: int
    params_per_device_eval_batch_size: int
    params_num_train_epochs: int
    params_weight_decay: float
    params_load_best_model_at_end: bool
    params_metric_for_best_model: str

In [50]:
from airTravelSentimentAnalysis.constants import *
from airTravelSentimentAnalysis.utils.common import read_yaml, create_directories

In [60]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        config["base_model_path"] = self.config.prepare_base_model.base_model_path
        config["base_tokenizer_path"] = self.config.prepare_base_model.base_tokenizer_path
        params_training = self.params.TRAINING_ARGUMENTS
        print(f"Training arguments: {params_training}")
        create_directories([config.root_dir])
        
        model_training_config = ModelTrainingConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            base_tokenizer_path=Path(config.base_tokenizer_path),
            model_path=Path(config.model_path),
            tokenizer_path=Path(config.tokenizer_path),
            train_tokenized_data_path=Path(config.train_tokenized_data_path),
            test_tokenized_data_path=Path(config.test_tokenized_data_path),
            val_tokenized_data_path=Path(config.val_tokenized_data_path),
            params_model_name=self.params.MODEL_NAME,
            params_evaluation_strategy=params_training.evaluation_strategy,
            params_save_strategy=params_training.save_strategy,
            params_learning_rate=params_training.learning_rate,
            params_per_device_train_batch_size=params_training.per_device_train_batch_size,
            params_per_device_eval_batch_size=params_training.per_device_eval_batch_size,
            params_num_train_epochs=params_training.num_train_epochs,
            params_weight_decay=params_training.weight_decay,
            params_load_best_model_at_end=params_training.load_best_model_at_end,
            params_metric_for_best_model=params_training.metric_for_best_model,
        )

        return model_training_config

In [61]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import EarlyStoppingCallback
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from datasets import load_from_disk
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
    
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits,axis=-1)  # Predicted class is the index of max logit
        precision = precision_score(labels, predictions, average="weighted")
        recall = recall_score(labels, predictions, average="weighted")
        f1 = f1_score(labels, predictions, average="weighted")
        accuracy = accuracy_score(labels, predictions)
        return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}
    
    def train(self):
        print(self.config)
        training_args = TrainingArguments(
            output_dir=self.config.model_path,eval_strategy ="epoch",
            save_strategy=self.config.params_save_strategy,
            # learning_rate=self.config.params_learning_rate,
            per_device_train_batch_size=self.config.params_per_device_train_batch_size,
            per_device_eval_batch_size=self.config.params_per_device_eval_batch_size,
            num_train_epochs=self.config.params_num_train_epochs,
            weight_decay=self.config.params_weight_decay,
            load_best_model_at_end=self.config.params_load_best_model_at_end,
            metric_for_best_model=self.config.params_metric_for_best_model
        )   
        model = AutoModelForSequenceClassification.from_pretrained(self.config.base_model_path)
        tokenizer = AutoTokenizer.from_pretrained(self.config.base_tokenizer_path)
        tokenized_train_dataset = load_from_disk(self.config.train_tokenized_data_path)
        tokenized_train_dataset = tokenized_train_dataset.rename_column("intent", "labels")
        tokenized_test_dataset = load_from_disk(self.config.test_tokenized_data_path)   
        tokenized_val_dataset = load_from_disk(self.config.val_tokenized_data_path)
        tokenized_val_dataset = tokenized_val_dataset.rename_column("intent", "labels")
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_train_dataset,
            eval_dataset=tokenized_val_dataset,
            tokenizer=tokenizer,
            compute_metrics=self.compute_metrics
        )
        trainer.train()
        trainer.save_model(self.config.model_path)
        tokenizer.save_pretrained(self.config.tokenizer_path)
    
  

In [62]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_training = ModelTraining(config=model_training_config)
    model_training.train()
except Exception as e:
    raise e

[2025-05-23 01:22:17,414: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-23 01:22:17,415: INFO: common: yaml file content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1taIeW6BZHqucmJbccEo92qfzZt5X_RTQ/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5', 'base_tokenizer_path': 'artifacts/prepare_base_tokenizer/base_model.h5', 'updated_base_tokenizer_path': 'artifacts/prepare_base_tokenizer/base_model_updated.h5'}, 'data_preprocessing': {'root_dir': 'artifacts/data_preprocessing', 'raw_data_file': 'artifacts/data_ingestion/bitext-travel-llm-chatbot-training-dataset.csv', 'train_data_path': 'artifa

C:\Users\assi01\AppData\Local\Temp\ipykernel_15824\3176620925.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 